# <center>Analyzing NYC Traffic Collision Data</center>
## <center>A 15-688 Project by:</center><center><br/> Ahmet Emre Unal (ahmetemu)<br/><br/>Marco Peyrot (mpeyrotc)</center>

In [ ]:
import pandas as pd
import numpy as np
import scipy
import collections

raw_data_file_name = 'NYPD_Motor_Vehicle_Collisions.csv'

In [10]:
def load_data(file_name):
    collision = pd.read_csv(file_name, 
                            na_filter=False, 
                            parse_dates={'DATE_COMBINED' : ['DATE', 'TIME']}, 
                            infer_datetime_format=True)
    
    # Remove rows that don't have the necessary data
    collision = collision[collision['LOCATION'] != '']
    collision = collision[collision['LATITUDE'] != '']
    collision = collision[collision['LONGITUDE'] != '']
    collision = collision[collision['BOROUGH'] != '']
    collision = collision[collision['ZIP CODE'] != '']
    collision = collision[collision['ON STREET NAME'] != '']
    collision = collision[collision['CROSS STREET NAME'] != '']
    collision = collision[collision['VEHICLE TYPE CODE 1'] != '']
    collision = collision[collision['VEHICLE TYPE CODE 1'] != 'UNKNOWN']

    # Drop unneeded columns
    columns_to_drop = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 
                       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 
                       'CONTRIBUTING FACTOR VEHICLE 5', 'LOCATION', 'UNIQUE KEY',
                       'OFF STREET NAME', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
                       'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
    for column in columns_to_drop:
        collision = collision.drop(column, axis=1)
        
    collision['ZIP CODE'] = collision['ZIP CODE'].astype(int)
    collision['LATITUDE'] = collision['LATITUDE'].astype(float)
    collision['LONGITUDE'] = collision['LONGITUDE'].astype(float)
        
    collision = collision.reset_index()
    
    return collision

# Load the collision data
collision = load_data(raw_data_file_name)
    
print collision.tail()
print collision.dtypes
print len(collision)

         index       DATE_COMBINED        BOROUGH  ZIP CODE   LATITUDE  \
606123  907038 2012-07-01 04:44:00      MANHATTAN     10001  40.746659   
606124  907040 2012-07-01 09:15:00       BROOKLYN     11233  40.675380   
606125  907041 2012-07-01 09:15:00         QUEENS     11421  40.696308   
606126  907042 2012-07-01 09:15:00         QUEENS     11429  40.711860   
606127  907043 2012-07-01 09:15:00  STATEN ISLAND     10310  40.634748   

        LONGITUDE    ON STREET NAME CROSS STREET NAME  \
606123 -73.990068          6 AVENUE    WEST 29 STREET   
606124 -73.908196    PACIFIC STREET   EASTERN PARKWAY   
606125 -73.851549         94 STREET           85 ROAD   
606126 -73.735538        219 STREET        104 AVENUE   
606127 -74.112253  CASTLETON AVENUE     BEMENT AVENUE   

        NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
606123                          1                         0   
606124                          0                         0   
606125                 

In [ ]:
print "hello"